In [1]:
#@title Imports

import numpy as np
import pandas as pd


import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re

import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

!pip install nlpaug
import nlpaug.augmenter.word as naw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 2.7 MB/s eta 0:00:00


In [2]:
#@title Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/MIDS-266/w266/project/Reddit/

both_test.csv	  both_train.gsheet  mh-all.csv      posts_train.csv  titles_train.csv
both_test.gsheet  both_val.csv	     mh-allout.csv   posts_val.csv    titles_val.csv
both_train.csv	  combo_train.csv    posts_test.csv  titles_test.csv


In [4]:
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="insert")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [5]:
import uuid
cdc_trainfile = '/content/drive/MyDrive/MIDS-266/w266/project/Reddit//mh-all.csv'
outfile = '/content/drive/MyDrive/MIDS-266/w266/project/Reddit//mh-allout.csv'
NUM_AUGMENTS = 3

orig_trainfile = '/content/drive/MyDrive/MIDS-266/w266/project/Reddit//both_train.csv'
combo_trainfile = '/content/drive/MyDrive/MIDS-266/w266/project/Reddit//combo_train.csv'

In [55]:


with open(outfile, 'w') as fw:
    fw.write('ID,title,post,class_name,class_id')
    with open(cdc_trainfile, 'r') as file:
        # Read all lines into a list
        lines = file.readlines()
        for line in lines:
            segs = line.split(',')
            class_id = segs[-1].strip()
            class_name = segs[-2]
            class_name = class_name.strip()
            topic = segs[0]
            body = segs[1:-2]
            body_string = " ".join(body)
            for i in range(NUM_AUGMENTS):
                augmented_text = aug.augment(body_string)
                uuid4 = uuid.uuid4()
                new_body = augmented_text[0]
                out = f'\n{uuid4}, {topic}, {new_body},{class_name}, {class_id}'
                print(out)
                fw.write(out)
            out = f'\n{uuid4}, {topic}, {body_string},{class_name}, {class_id}'
            print(out)
            fw.write(out)




8e5ac38f-483e-4697-962f-e9af20ac73cf, CDC definition, " miss personal details or otherwise make careless petty mistakes intervening in schoolwork at work earlier or interfering in other activities ",adhd, 0

eb1e205b-dfd9-4c5e-9993-24199b6052bf, CDC definition, " may miss details or accidentally make careless math mistakes in schoolwork class at work unless or clumsy in other activities ",adhd, 0

452f3f12-e358-45b6-95d1-a4ceac345469, CDC definition, " miss the details or make more careless mistakes if in doing schoolwork at work or in conducting other human activities ",adhd, 0

452f3f12-e358-45b6-95d1-a4ceac345469, CDC definition,  "Miss details or make careless mistakes in schoolwork  at work  or in other activities",adhd, 0

ec745cd9-97b1-4e99-a30f-7a6b6d9ad9ba, CDC definition, " have trouble always staying strictly focused on their play activities and or work tasks ",adhd, 0

c08cb92a-f13c-4482-b914-4eefc8fbfd40, CDC definition, " have any trouble at staying focused... on play ac

Let's load this content to a pandas dataframe and inspect the class balance

In [57]:
cdc_df = pd.read_csv(outfile)

In [58]:
cdc_df.class_name.value_counts()

class_name
adhd          96
anxiety       96
bipolar       96
depression    96
ptsd          96
Name: count, dtype: int64

In [59]:
classes = cdc_df.class_name.unique()
list(classes)

['adhd', 'anxiety', 'bipolar', 'depression', 'ptsd']

Load the original training dataset into a different dataframe

In [60]:
orig_df = pd.read_csv(orig_trainfile)

In [61]:
orig_df.class_name.value_counts()

class_name
adhd          2465
depression    2450
anxiety       2422
bipolar       2407
ptsd          2001
none          1982
Name: count, dtype: int64

To balance the dataset, let's extract 96 entries from each class and 192 from the 'none' class

In [63]:
newdf = cdc_df

tempdf = orig_df.query("class_name == 'adhd'")
newdf = pd.concat([newdf, tempdf[0:96]], ignore_index=True)

tempdf = orig_df.query("class_name == 'anxiety'")
newdf = pd.concat([newdf, tempdf[0:96]], ignore_index=True)

tempdf = orig_df.query("class_name == 'depression'")
newdf = pd.concat([newdf, tempdf[0:96]], ignore_index=True)

tempdf = orig_df.query("class_name == 'ptsd'")
newdf = pd.concat([newdf, tempdf[0:96]], ignore_index=True)

tempdf = orig_df.query("class_name == 'bipolar'")
newdf = pd.concat([newdf, tempdf[0:96]], ignore_index=True)

tempdf = orig_df.query("class_name == 'none'")
newdf = pd.concat([newdf, tempdf[0:192]], ignore_index=True)



In [64]:
newdf

,ID,title,post,class_name,class_id
0,8e5ac38f-483e-4697-962f-e9af20ac73cf,CDC definition,""" miss personal details or otherwise make car...",adhd,0
1,eb1e205b-dfd9-4c5e-9993-24199b6052bf,CDC definition,""" may miss details or accidentally make carel...",adhd,0
2,452f3f12-e358-45b6-95d1-a4ceac345469,CDC definition,""" miss the details or make more careless mist...",adhd,0
3,452f3f12-e358-45b6-95d1-a4ceac345469,CDC definition,"""Miss details or make careless mistakes in s...",adhd,0
4,ec745cd9-97b1-4e99-a30f-7a6b6d9ad9ba,CDC definition,""" have trouble always staying strictly focuse...",adhd,0
...,...,...,...,...,...
1147,c204559d-974b-4d1f-b54a-1a8e4b0430cf,reverse engineering a bézier curve,i have a bézier curve for which i know the coo...,none,5
1148,fd428ac9-4418-4de9-9dee-24333bf76d20,why isn't election day - arguably the single m...,[christopher columbus day is a national holida...,none,5
1149,28b37f96-9905-44ba-92ce-45d60abce6e8,i think taylor swift stole this dude's song...,"so i found this guy on spotify ""matthew perry ...",none,5
1150,beb31510-720a-4ac0-8096-4052c45c8aff,"i’m rick astley, and you can ask me anything, ...","hey reddit, since my last ama i’ve had a burni...",none,5


In [65]:
newdf.to_csv(combo_trainfile, index=False)

In [66]:
newdf.class_name.value_counts()

class_name
adhd          192
anxiety       192
bipolar       192
depression    192
ptsd          192
none          192
Name: count, dtype: int64